# GenMol Demo

In [16]:
!pip install rdkit python-dotenv pandas numpy matplotlib loguru py3dmol


In [17]:
import pickle
from typing import List
import numpy as np

from rdkit.Chem.QED import qed as rdkit_qed
from rdkit import Chem

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from rdkit.Chem.QED import qed
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.DataStructs import TanimotoSimilarity
import ast
from google.colab import userdata
import os, shutil, requests
import pandas as pd

In [72]:
API_KEY = userdata.get('API_KEY')
# print(API_KEY)

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json",
}

invoke_url = "https://health.api.nvidia.com/v1/biology/nvidia/genmol/generate"

session = requests.Session()

In [73]:
smis = "CC[*]"

In [75]:

# Create the request payload
payload = {
    "smiles": smis,
    "num_molecules": "30",
    "temperature": "1",
    "noise": "0",
    "step_size":"1",
    "scoring": "QED"
}

# Send the request and get the response
response = session.post(invoke_url, headers=headers, json=payload)
response.raise_for_status()
result = response.json()

print(f"*************** RESPONSE ********************")
print(f"result: \n"
  f"{result}")



HTTPError: 400 Client Error: Bad Request for url: https://health.api.nvidia.com/v1/biology/nvidia/genmol/generate

In [52]:
smis = "CC(C)CN(Cc1ccc([*])cc1)[S](=O)(=O)Cc2ccccc2"

In [ ]:
smis = "CC(C)CN(Cc1ccc(cc1)N2CCN(CC2)C(C)=O)[S](=O)(=O)Cc3ccccc3"

mol = Chem.MolFromSmiles(smis)
qed_score = rdkit_qed(mol)
print(f"Original QED: {qed_score}")

# Generate an image from the molecule
img = Draw.MolToImage(mol, size=(300, 300))
display(img)  # this will display the SM image in a Jupyter Notebook

Removing the acyl-piperazine to facilitate exploration of alterantive moeities